# Рекурентные сети для обработки последовательностей

В данной теме надо будет:

* Попробуем обучить нейронную сеть GRU/LSTM для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech
* Сделать выводы

In [42]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from string import punctuation
from stop_words import get_stop_words
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# lemmatizer
from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem

In [34]:
MAX_WORDS = 2000
MAX_LEN = 20
NUM_CLASSES = 1

# Training
EPOCHS = 5
BATCH_SIZE = 512
PRINT_BATCH_N = 100

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

SW = set(get_stop_words('english'))
PUNCTS = set(punctuation)

TYPE_LEMM = 'WordNetLemmatizer'

tqdm.pandas()
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')

# Для эксперемента используем несколько библиотек лемматизации
wn_lemmatizer = WordNetLemmatizer()
mystem_lemmatizer = Mystem()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vlad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vlad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vlad\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [36]:
df_train = pd.read_csv("data/GRU_LSTM/train.csv")

### Функции, которые нам понадобятся

In [37]:
def lemmatizer_text(txt, TYPE_LEMM):
    result_txt = []
    for word in txt.split():
        if word in SW:
            continue
        if TYPE_LEMM == 'WordNetLemmatizer':
            result_txt.append(wn_lemmatizer.lemmatize(word))
        elif TYPE_LEMM == 'Mystem':
            m_l = mystem_lemmatizer.lemmatize(word)
            if (len(m_l) > 0):
                result_txt.append(m_l[0])
    return result_txt

# Функция для предобработки текста
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in PUNCTS)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = re.sub(r"@[\w]*", "", txt)
    txt = re.sub(r'[^\w\s]', " ", txt)
    txt = re.sub(r"[^a-zA-Z0-9]"," ", txt)
    txt = re.sub(r"[^a-zA-Z0-9]"," ", txt)
    txt = lemmatizer_text(txt, TYPE_LEMM)
    return " ".join(txt)

# Функция для сборки последовательности
def text_to_sequence(text, maxlen, vocabulary):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

### Опишем нейронную сеть (с учетом выбора типа модели) и Dataset

In [38]:
#
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)
#
class Net(nn.Module):
    def __init__(self, vocab_size=MAX_WORDS, type_model='rnn', embedding_dim=128, hidden_dim=128, use_last=True) -> None:
        super().__init__()
        self.use_last = use_last
        self.type_model = type_model
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if type_model == 'rnn':
            self.model = nn.RNN(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        elif type_model == 'lstm':
            self.model = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        elif type_model == 'gru':
            self.model = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)

        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        output = self.embeddings(x)
        output = self.dropout(output)

        model_out, ht = self.model(output)
        if self.use_last:
            last_tensor = model_out[:,-1,:]
        else:
            # use mean
            last_tensor = torch.mean(model_out[:,:], dim=1)
        output = self.linear(last_tensor)
        output = torch.sigmoid(output)
        return output

### Предобаботаем данныые

In [50]:
for TYPE_LEMM in ['WordNetLemmatizer']:
    print(f"Preprocess for lemma: {TYPE_LEMM}")
    # предобрабатываем текст
    df_train['pre_proc_tweet_' + TYPE_LEMM] = df_train['tweet'].progress_apply(preprocess_text)

    print(f"Break token for lemma: {TYPE_LEMM}")
    # разбиваем на токены
    tokens = word_tokenize(" ".join(df_train['pre_proc_tweet_' + TYPE_LEMM]))
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for (MAX_WORDS, MAX_LEN) in [(10, 5), (100, 10), (1000, 100), (10000, 100)]:
        # создаем словарь
        dist = FreqDist(tokens_filtered)
        tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]  # вычитание 1 для padding
        vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

        x_train = np.asarray([text_to_sequence(text, MAX_LEN, vocabulary) for text in df_train['pre_proc_tweet_' + TYPE_LEMM]])

        # Загружаем данные в dataset
        train_dataset = DataWrapper(x_train, df_train['label'].values)
        # Разбиваем наши данные на тренеровочные и тестовые с параметрамми для трейна = 67% и теста = 33%
        train_dataset, test_dataset = train_test_split(train_dataset, train_size=0.67, test_size=0.33)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

        for use_last in [True, False]:
            for type_model in ['rnn', 'lstm', 'gru']:

                # создаем нашу модель
                model = Net(vocab_size=MAX_WORDS, type_model=type_model, embedding_dim=(MAX_LEN-1), use_last=use_last)

                for optim in ['SGD', 'Adam']:
                    # Оптимизатор и функция потерь
                    if optim == 'SGD':
                        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
                    else:
                        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

                    criterion = nn.BCELoss()

                    model = model.to(DEVICE)
                    th = 0.5

                    train_loss_history = []
                    test_loss_history = []

                    for epoch in range(EPOCHS):
                        model.train()
                        running_loss, running_items, running_right = 0.0, 0.0, 0.0
                        for i, train_data in enumerate(train_loader, 0):
                            inputs, labels = train_data[0].to(DEVICE), train_data[1].to(DEVICE)

                            # обнуляем градиент
                            optimizer.zero_grad()
                            outputs = model(inputs)

                            loss = criterion(outputs, labels.float().view(-1, 1))
                            loss.backward()
                            optimizer.step()

                            # подсчет ошибки на обучении
                            loss = loss.item()
                            running_items += len(labels)
                            # подсчет метрики на обучении
                            pred_labels = torch.squeeze((outputs > th).int())
                            running_right += (labels == pred_labels).sum()

                            # выводим статистику о процессе обучения
                            if i % 150 == 0:    # печатаем каждые 150 batches
                                model.eval()

                                # f1_score_train = f1_score(labels, pred_labels, zero_division=1)
                                # precision_score_train = precision_score(labels, pred_labels, zero_division=1)
                                # recall_score_train = recall_score(labels, pred_labels, zero_division=1)

                                print(f'Epoch [{epoch + 1}/{EPOCHS}]. ' +
                                      f'Step [{i + 1}/{len(train_loader)}]. ' +
                                      f'Loss: {loss:.3f}. ' +
                                      f'Acc: {running_right / running_items:.3f}', end='. ')
                                      # f'f1_score: {f1_score_train:.3f}. ' +
                                      # f'precision_score: {precision_score_train:.3f}. ' +
                                      # f'recall_score: {recall_score_train:.3f}.'

                                running_loss, running_items, running_right = 0.0, 0.0, 0.0
                                train_loss_history.append(loss)

                                # выводим статистику на тестовых данных
                                test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
                                for j, test_data in enumerate(test_loader):
                                    test_labels = test_data[1]
                                    test_outputs = model(test_data[0])

                                    # подсчет ошибки на тесте
                                    test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
                                    # подсчет метрики на тесте
                                    test_running_total += len(test_data[1])
                                    pred_test_labels = torch.squeeze((test_outputs > th).int())
                                    test_running_right += (test_labels == pred_test_labels).sum()

                                    # f1_score_test = f1_score(test_labels, pred_test_labels, zero_division=1)
                                    # precision_score_test = precision_score(test_labels, pred_test_labels, zero_division=1)
                                    # recall_score_test = recall_score(test_labels, pred_test_labels, zero_division=1)

                                test_loss_history.append(test_loss.item())
                                print(f'Test loss: {test_loss:.3f}. '
                                      f'Test acc: {test_running_right / test_running_total:.3f}.')
                                      # f'Test f1_score_test: {f1_score_test}. '
                                      # f'Test precision_score: {precision_score_test}. '
                                      # f'Test recall_score_test: {recall_score_test}')

                            model.train()

                    print(f'Training for lem: {TYPE_LEMM}, MAX_WORDS: {MAX_WORDS}, MAX_LEN: {MAX_LEN}, optim: {optim}, type_model: {type_model}, use_last: {str(use_last)}  finished!')

Preprocess for lemma: WordNetLemmatizer


100%|██████████| 31962/31962 [00:01<00:00, 21110.69it/s]


Break token for lemma: WordNetLemmatizer
Epoch [1/5]. Step [1/42]. Loss: 0.630. Acc: 0.938. Test loss: 0.646. Test acc: 0.929.
Epoch [2/5]. Step [1/42]. Loss: 0.361. Acc: 0.941. Test loss: 0.559. Test acc: 0.929.
Epoch [3/5]. Step [1/42]. Loss: 0.292. Acc: 0.920. Test loss: 0.615. Test acc: 0.929.
Epoch [4/5]. Step [1/42]. Loss: 0.248. Acc: 0.934. Test loss: 0.675. Test acc: 0.929.
Epoch [5/5]. Step [1/42]. Loss: 0.259. Acc: 0.928. Test loss: 0.702. Test acc: 0.929.
Training for lem: WordNetLemmatizer, MAX_WORDS: 10, MAX_LEN: 5, optim: SGD, type_model: rnn, use_last: True  finished!
Epoch [1/5]. Step [1/42]. Loss: 0.254. Acc: 0.930. Test loss: 1.831. Test acc: 0.929.
Epoch [2/5]. Step [1/42]. Loss: 0.218. Acc: 0.939. Test loss: 0.766. Test acc: 0.929.
Epoch [3/5]. Step [1/42]. Loss: 0.256. Acc: 0.930. Test loss: 0.780. Test acc: 0.929.
Epoch [4/5]. Step [1/42]. Loss: 0.253. Acc: 0.926. Test loss: 0.663. Test acc: 0.929.
Epoch [5/5]. Step [1/42]. Loss: 0.216. Acc: 0.938. Test loss: 0.84

## Вывод

Было проведено 48 тестрирований (результаты представлены на результате 5-ой эпохи), для лемматизации использовался - WordNetLemmatizer:

С вариациями:

1) Изменение количества максимальных слов: 10, 100, 1000, 100000 и максимальную длинну: 5, 10, 100, 100

2) Изменение метода оптимизатора: SGD, Adam

3) Изменение разновидности нейронных сетей: rnn, lstm, gru

4) Использовать результат предыдущего вычисления: Tru, False

**Результат**:

Сразу оговорюсь, результат учитывался именно в последней эпохи.

Наилучшим вариант был при варианте: количества максимальных слов (MAX_WORDS) - 10000, максимальной длинны (MAX_LEN) - 100, оптимизатор: Adam, тип нейронной сети: gru, использование предыдущего результата: False.

Худшими вариантами были при вариантах: количества максимальных слов (MAX_WORDS) - 1000, максимальной длинны (MAX_LEN) - 100, оптимизатор: SGD и Adam,
тип нейронной сети: rnn, lstm, gru, использование предыдущего результата: True, False.

**Вывод**

Похоже для обучение нейронной сети посредством rnn, lstm, gru лучше использовать побольшей количества слов и оптимизатор Adam.

P.S. были попытки использовать леммитизатор от бибилиотеки: pymystem3, но его скорость сильно уступает WordNetLemmatizer, потому в последствии пришлось отказаться от него.